# BNNs

In [1]:
from bayesian_models import Pbnn
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from doepy import build
import pickle
from scipy.stats import norm, uniform, lognorm

In [2]:
from four_branch import four_branch

function = four_branch

def convert_lognormal(mean_ln, std_ln):
    gaussian_param = np.zeros(2)

    SigmaLogNormal = np.sqrt( np.log(1+(std_ln/mean_ln)**2))
    MeanLogNormal = np.log( mean_ln ) - SigmaLogNormal**2/2

    gaussian_param[0] = MeanLogNormal
    gaussian_param[1] = SigmaLogNormal

    return gaussian_param

Active Train

In [3]:
dim = 2
n_mcs = int(1e6)
x_test = np.random.normal(0, 1, size=(n_mcs, dim))

exp = {'x1': [0, 1, 'normal'],
       'x2': [0, 1, 'normal']}

y_test = function(x_test)
Pf_ref = np.sum( y_test < 0 ) / n_mcs
B_ref= - norm.ppf( Pf_ref )
Pf_ref, B_ref

(0.004372, 2.6219042243725723)

In [4]:
dim = 2
output = 1
layers, archit = 2 , [8, 8] #ange archit size if layers are increased

passive_samples = 10


#PASSIVE TRAINING---------------------------------------------------
X = np.zeros((passive_samples, dim))

exp_norm = {}
for var_name in range(dim):
    exp_norm['x'+ str(var_name+1)] = [0.000001, 0.999999]    #initial design domain for each variable (normal, uniform)
#Latin hypercube sampling
Xdoe = build.space_filling_lhs(exp_norm , num_samples = passive_samples)

for margin in range (0, dim):
    var = 'x' + str (margin + 1)

    if exp[var][2] == 'normal':
        loc_ = exp[var][0]
        scale_ = exp[var][1]
        X[:, margin] = norm.ppf(Xdoe[var], loc=loc_, scale=scale_)

    elif exp[var][2] == 'uniform':
        loc_ = exp[var][0]
        scale_ = exp[var][1]
        X[:, margin] = uniform.ppf(Xdoe[var], loc=loc_, scale=scale_-loc_)

    elif exp[var][2] == 'lognormal':
        xlog_mean = exp[var][0]
        xlog_std = exp[var][1]
        gaussian_param = convert_lognormal(xlog_mean, xlog_std)
        X[:, margin] = lognorm.ppf(Xdoe[var], s=gaussian_param[1], scale=xlog_mean) 

Y = function(X)

n_doe = len(X)
#-------------------------------------------creating bnn
#setting up the network architecture -----------------------------------
config = {"n_infeatures": dim,
        "n_outfeatures": output,
        "n_samples": n_doe,
        "learn_all_params": False,  #to learn mean and sigma
        "fixed_param": 0.001} 

ModelName = 'BNN_' + str(n_doe)
mybnn = Pbnn(config)

mybnn.build_bnn(layers, archit) #----------------------------------------------------------MODEL ARCHITECTURE

C:\Users\Administrator\anaconda3\envs\deep_learning\lib\site-packages\tensorflow_probability\python\layers\util.py:99: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
C:\Users\Administrator\anaconda3\envs\deep_learning\lib\site-packages\tensorflow_probability\python\layers\util.py:109: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  untransformed_scale = add_variable_fn(


In [39]:
#ACTIVE TRAIN LOOP ---------------------------------------------------
training_epochs = 15000
batch_size = len(X)
# batch_size = [np.floor_divide(len(X), 2)][0] + 1  # half of the DoE

#-------------------------------------------training bnn
train_env = {"batch_size": batch_size,
            "learning_rate": 0.001,
            "epochs": training_epochs,
            "callback_patience": 4000,
            "verbose": 1}
print('Training BNN With', len(X), 'samples' )
mybnn.train_bnn(X, Y, train_env)

Training BNN With 16 samples
Training completed
Minimum loss:  2860.498779296875


In [40]:
n_mcs = int(1e6)
bnn_simulations = 500

#-------------------------------------------MC population
Xtest = np.zeros((int(n_mcs), dim))
MCinputs_norm= np.random.uniform(0.000001, 0.999999, size=(int(n_mcs), dim))

##Iso-probabilistic transformation-------------------------------------------
for margin in range (0, dim):
    var = 'x' + str (margin + 1)

    if exp[var][2] == 'normal':
        loc_ = exp[var][0]
        scale_ = exp[var][1]
        Xtest[:, margin] = norm.ppf(MCinputs_norm[:, margin], loc=loc_, scale=scale_)

    elif exp[var][2] == 'uniform':
        loc_ = exp[var][0]
        scale_ = exp[var][1]
        Xtest[:, margin] = uniform.ppf(MCinputs_norm[:, margin], loc=loc_, scale=scale_-loc_)

    elif exp[var][2] == 'lognormal':
        xlog_mean = exp[var][0]
        xlog_std = exp[var][1]
        gaussian_param = convert_lognormal(xlog_mean, xlog_std)
        Xtest[:, margin] = lognorm.ppf(MCinputs_norm[:, margin], s=gaussian_param[1], scale=xlog_mean) 

#-------------------------------------------model predictions over MC population
print('BNN predictions with MC population...')
Mean_muY_MC, Stdv_muY_MC, Mean_sigmaY_MC, Stdv_sigmaY_MC = mybnn.modeluq_bnn(Xtest, nsim = bnn_simulations)
y_mcs = function(Xtest)

PF = np.sum(Mean_muY_MC < 0) / n_mcs
PF_ref = np.sum(y_mcs < 0) / n_mcs
B = - norm.ppf( PF )

print('PF_ref =', PF_ref,'PF =', PF, 'and B =',"%.5f" % round(B, 3))

BNN predictions with MC population...
PF_ref = 0.004434 PF = 0.002098 and B = 2.86300


In [36]:
Mean_muY_val, Stdv_muY_val, Mean_sigmaY_val, Stdv_sigmaY_val = mybnn.modeluq_bnn(X, nsim = bnn_simulations)
Mean_muY_val.reshape(-1) - Y

array([ 0.07487402, -0.008905  , -0.00747536, -0.00590837,  0.08573978,
       -0.00523244, -0.00342153, -0.00150835, -0.00515859,  0.08822284,
        0.05993528, -0.16480754, -0.00171553, -0.00483883, -0.22874546])

In [38]:
#-------------------------------------------Selecting new training point
U_f = np.abs(Mean_muY_MC) / Stdv_muY_MC
U_min = np.argmin(U_f)
X_new = Xtest[U_min].reshape(-1, dim)
X = np.concatenate((X, X_new), axis=0)

Y_new = function(X_new)
Y = np.concatenate((Y, Y_new), axis=0)
print(Y)

[ 2.6398568   2.5912778  -1.2754917   2.6051097   1.894706    1.801368
  1.9185399  -0.08005714  2.8024201  -3.7223573   0.5917287   0.3365314
  0.33764505  1.5427594   0.46241045 -0.00950265]


In [8]:
print('PF =', PF, 'and B =',"%.5f" % round(B, 3))
len(X)

PF = 0.009501 and B = 2.34500


10

In [12]:
print('PF =', PF, 'and B =',"%.5f" % round(B, 3))
len(X)

PF = 0.004975 and B = 2.57800


11

In [17]:
print('PF =', PF, 'and B =',"%.5f" % round(B, 3))
len(X)

PF = 0.005231 and B = 2.56000


12

In [24]:
print('PF =', PF, 'and B =',"%.5f" % round(B, 3))
len(X)

PF = 0.003926 and B = 2.65800


13

In [28]:
print('PF =', PF, 'and B =',"%.5f" % round(B, 3))
len(X)

PF = 0.00218 and B = 2.85100


14

In [35]:
print('PF =', PF, 'and B =',"%.5f" % round(B, 3))
len(X)

PF = 0.00225 and B = 2.84100


15

In [41]:
print('PF =', PF, 'and B =',"%.5f" % round(B, 3))
len(X)

PF = 0.002098 and B = 2.86300


16

In [ ]:
print('PF =', PF, 'and B =',"%.5f" % round(B, 3))
len(X)

In [ ]:
mybnn.train_bnn(X_new, Y_new, train_env)